Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A2 - Wikipedia, ORES, and Bias in Data
Please follow the reproducability workflow as practiced during the last exercise.

## Step 1⃣ | Data acquisition

You will use two data sources: (1) Wikipedia articles of politicians and (2) world population data.

**Wikipedia articles -**
The Wikipedia articles can be found on [Figshare](https://figshare.com/articles/Untitled_Item/5513449). It contains politiciaans by country from the English-language wikipedia. Please read through the documentation for this repository, then download and unzip it to extract the data file, which is called `page_data.csv`.

**Population data -**
The population data is available in `CSV` format in the `_data` folder. The file is named `export_2019.csv`. This dataset is drawn from the [world population datasheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau (downloaded 2020-11-13 10:14 AM). I have edited the dataset to make it easier to use in this assignment. The population per country is given in millions!

In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter

In [2]:
population_df = pd.read_csv("../data_raw/export_2019.csv", sep=";")
population_df.head()

,country,population,region
0,Algeria,44.357,AFRICA
1,Egypt,100.803,AFRICA
2,Libya,6.891,AFRICA
3,Morocco,35.952,AFRICA
4,Sudan,43.849,AFRICA


In [3]:
politician_df = pd.read_csv("../data_raw/page_data.csv", sep=",")
politician_df.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


## Step 2⃣ | Data processing and cleaning
The data in `page_data.csv` contain some rows that you will need to filter out. It contains some page names that start with the string `"Template:"`. These pages are not Wikipedia articles, and should not be included in your analysis. The data in `export_2019.csv` does not need any cleaning.

***

| | `page_data.csv` | | |
|-|------|---------|--------|
| | **page** | **country** | **rev_id** |
|0|	Template:ZambiaProvincialMinisters | Zambia | 235107991 |
|1|	Bir I of Kanem | Chad | 355319463 |

***

| | `export_2019.csv` | | |
|-|------|---------|--------|
| | **country** | **population** | **region** |
|0|	Algeria | 44.357 | AFRICA |
|1|	Egypt | 100.803 | 355319463 |

***

### Remove page names that contain "Template"

In [4]:
politician_df = politician_df[~politician_df["page"].str.contains("Template")]
politician_df.head()

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568


In [5]:
#politician_df.to_csv("../data_clean/en-wikipedia-politicians.csv", index=False)

In [6]:
#population_df.to_csv("../data_clean/prb-world-population.csv", index=False)

### Getting article quality predictions with ORES

Now you need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called [**ORES**](https://www.mediawiki.org/wiki/ORES) ("Objective Revision Evaluation Service"). ORES estimates the quality of an article (at a particular point in time), and assigns a series of probabilities that the article is in one of the six quality categories. The options are, from best to worst:

| ID | Quality Category |  Explanation |
|----|------------------|----------|
| 1 | FA    | Featured article |
| 2 | GA    | Good article |
| 3 | B     | B-class article |
| 4 | C     | C-class article |
| 5 | Start | Start-class article |
| 6 | Stub  | Stub-class article |

For context, these quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. If you're curious, you can [read more](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment#Grades) about what these assessment classes mean on English Wikipedia. For this assignment, you only need to know that these categories exist, and that ORES will assign one of these six categories to any `rev_id`. You need to extract all `rev_id`s in the `page_data.csv` file and use the ORES API to get the predicted quality score for that specific article revision.

### ORES REST API endpoint

The [ORES REST API](https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model) is configured fairly similarly to the pageviews API we used for the last assignment. It expects the following parameters:

* **project** --> `enwiki`
* **revid** --> e.g. `235107991` or multiple ids e.g.: `235107991|355319463` (batch)
* **model** --> `wp10` - The name of a model to use when scoring.

**❗Note on batch processing:** Please read the documentation about [API usage](https://www.mediawiki.org/wiki/ORES#API_usage) if you want to query a large number of revisions (batches). 

You will notice that ORES returns a prediction value that contains the name of one category (e.g. `Start`), as well as probability values for each of the six quality categories. For this assignment, you only need to capture and use the value for prediction.

**❗Note:** It's possible that you will be unable to get a score for a particular article. If that happens, make sure to maintain a log of articles for which you were not able to retrieve an ORES score. This log should be saved as a separate file named `ORES_no_scores.csv` and should include the `page`, `country`, and `rev_id` (just as in `page_data.csv`).

### Method to query ORES predictions

In [7]:
# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/chrisk280',
    'From': 'chrisk31@zedat.fu-berlin.de'
}

def get_ores_data(rev_id_list, headers):
    
    # Define the endpoint
    # https://ores.wikimedia.org/scores/enwiki/?models=wp10&revids=807420979|807422778
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : '|'.join([str(x) for x in rev_id_list])
              }
    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    data = json.dumps(response)

    return data

Sending one request for each `rev_id` might take some time. If you want to send batches you can use `'|'.join(str(x) for x in revision_ids` to put your ids together. Please make sure to deal with [exception handling](https://www.w3schools.com/python/python_try_except.asp) of the `KeyError` exception, when extracting the `prediction` from the `JSON` response.

In [8]:
#politician_df = pd.read_csv("../data_clean/en-wikipedia-politicians.csv")
#population_df = pd.read_csv("../data_clean/prb-world-population.csv")

In [9]:
all_rev_ids = politician_df["rev_id"].to_list()
len(all_rev_ids)

46701

### Get ORES data

#### Get example data to check the structure

In [10]:
politician_df["pred"] = np.nan
politician_df.head()

,page,country,rev_id,pred
1,Bir I of Kanem,Chad,355319463,NaN
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,NaN
12,Yos Por,Cambodia,393822005,NaN
23,Julius Gregr,Czech Republic,395521877,NaN
24,Edvard Gregr,Czech Republic,395526568,NaN


In [11]:
get_ores_data(all_rev_ids[0:2], headers=headers)

'{"enwiki": {"models": {"wp10": {"version": "0.8.2"}}, "scores": {"355319463": {"wp10": {"score": {"prediction": "Stub", "probability": {"B": 0.005643168767502225, "C": 0.005641424870624224, "FA": 0.0010757577110297029, "GA": 0.001543343686495854, "Start": 0.010537503531047517, "Stub": 0.9755588014333005}}}}, "393276188": {"wp10": {"score": {"prediction": "Stub", "probability": {"B": 0.008182590350064387, "C": 0.00889480723831463, "FA": 0.0013360623544989905, "GA": 0.002367361886443145, "Start": 0.0455054831381552, "Stub": 0.9337136950325238}}}}}}}'

#### Function to query ORES API and extract the predictions

In [12]:
def get_predictions(rev_ids, headers):
    predictions = []
    data = json.loads(get_ores_data(rev_ids, headers))
    for key, value in data["enwiki"]["scores"].items():
        try:
            predictions.append(value["wp10"]["score"]["prediction"])
        except KeyError:
            predictions.append(np.nan)
    return predictions
    

### Query ORES predictions for every article in batches of 50

In [ ]:
batch_size = 50
for i in range(0, len(all_rev_ids), batch_size):
    if i == 0 or (1+(i/batch_size))%20 == 0:
        print("Processing batch {} of {}".format(int(1+(i/batch_size)), int(np.ceil(len(all_rev_ids)/batch_size))))
    # determine size of the next batch; the last batch can be smaller than 50 rev_ids
    if i+batch_size > len(all_rev_ids):
        batch = all_rev_ids[i:]
    else:
        batch = all_rev_ids[i:i+batch_size]
    # query API
    batch_pred = get_predictions(batch, headers)
    # add predictions to politician_df
    politician_df.iloc[i:i+len(batch_pred),3] = batch_pred
print("All batches processed")

Processing batch 1 of 935
Processing batch 20 of 935
Processing batch 40 of 935
Processing batch 60 of 935
Processing batch 80 of 935
Processing batch 100 of 935
Processing batch 120 of 935
Processing batch 140 of 935
Processing batch 160 of 935
Processing batch 180 of 935
Processing batch 200 of 935
Processing batch 220 of 935
Processing batch 240 of 935
Processing batch 260 of 935
Processing batch 280 of 935
Processing batch 300 of 935
Processing batch 320 of 935
Processing batch 340 of 935
Processing batch 360 of 935
Processing batch 380 of 935
Processing batch 400 of 935
Processing batch 420 of 935
Processing batch 440 of 935
Processing batch 460 of 935
Processing batch 480 of 935
Processing batch 500 of 935
Processing batch 520 of 935


#### Inspect the resulting DataFrame with predictions

In [ ]:
politician_df.head(3)

### Write articles without a prediction into a file

In [ ]:
politician_no_pred_df = politician_df[politician_df["pred"].isna()]
politician_no_pred_df.head(2)

In [ ]:
politician_no_pred_df.drop(axis=1, labels=["pred"]).to_csv("../data_clean/ORES_no_scores.csv", index=False)

### Drop articles without a prediction

In [ ]:
politician_df = politician_df[~politician_df["pred"].isna()]

In [ ]:
#### Write the results into a file so we do not have to query the API again

In [ ]:
politician_df.to_csv("../data_clean/en-wikipedia-politicians_with_article_quality.csv", index=False)

## Combining the datasets

Now you need to combine both dataset: (1) the wikipedia articles and its ORES quality scores and (2) the population data. Both have columns named `country`. After merging the data, you'll invariably run into entries which cannot be merged. Either the population dataset does not have an entry for the equivalent Wikipedia country, or vis versa.

Please remove any rows that do not have matching data, and output them to a `CSV` file called `countries-no_match.csv`. Consolidate the remaining data into a single `CSV` file called `politicians_by_country.csv`.

The schema for that file should look like the following table:


| article_name | country | region | revision_id | article_quality | population |
|--------------|---------|--------|-------------|-----------------|------------|
| Bir I of Kanem | Chad  | AFRICA | 807422778 | Stub | 16877000 |

### Remove countries that only exist in one of the datasets and can't be merged on

In [ ]:
politician_df = pd.read_csv("../data_clean/en-wikipedia-politicians_with_article_quality.csv")
population_df = pd.read_csv("../data_clean/prb-world-population.csv")

In [ ]:
politician_df["country"] = politician_df["country"].str.lower()
population_df["country"] = population_df["country"].str.lower()

In [ ]:
c_no_match = set(politician_df["country"]).symmetric_difference(set(population_df["country"]))
len(c_no_match)

In [ ]:
pd.DataFrame({"country": list(c_no_match)}).to_csv("../data_clean/politicians_by_country.csv", index=False, sep=";")

In [ ]:
population_df = population_df[~population_df["country"].isin(c_no_match)]
politician_df = politician_df[~politician_df["country"].isin(c_no_match)]

### Merge population and politician DataFrames

In [ ]:
pol_pop_df = population_df.merge(politician_df, on="country", how="outer")
pol_pop_df.head(3)

#### Reorder and Rename columns

In [ ]:
pol_pop_df.columns = ["country", "population", "region", "article_name", "revision_id", "article_quality"]
pol_pop_df = pol_pop_df[["article_name", "country", "region", "revision_id", "article_quality", "population"]]
pol_pop_df.columns

### Write cleaned up data into file

In [ ]:
pol_pop_df.to_csv("../data_clean/politicians_by_country.csv", index=False)

#### Small checks to see if everything went correctly

In [ ]:
pol_pop_df[pol_pop_df["pred"].isna()]

In [ ]:
pol_pop_df[pol_pop_df["country"].isna()]

## Step 3⃣ | Analysis

Your analysis will consist of calculating the proportion (as a percentage) of articles-per-population (we can also call it `coverage`) and high-quality articles (we can also call it `relative-quality`)for **each country** and for **each region**. By `"high quality"` arcticle we mean an article that ORES predicted as `FA` (featured article) or `GA` (good article).

**Examples:**

* if a country has a population of `10,000` people, and you found `10` articles about politicians from that country, then the percentage of `articles-per-population` would be `0.1%`.
* if a country has `10` articles about politicians, and `2` of them are `FA` or `GA` class articles, then the percentage of `high-quality-articles` would be `20%`.

### Results format

The results from this analysis are six `data tables`. Embed these tables in the Jupyter notebook. You do not need to graph or otherwise visualize the data for this assignment. The tables will show:

1. **Top 10 countries by coverage**<br>10 highest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Bottom 10 countries by coverage**<br>10 lowest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Top 10 countries by relative quality**<br>10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Bottom 10 countries by relative quality**<br>10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population
1. **Regions by relative quality**<br>Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

**❗Hint:** You will find what country belongs to which region (e.g. `ASIA`) also in `export_2019.csv`. You need to calculate the total poulation per region. For that you could use `groupby` and also check out `apply`.

In [ ]:
pp_df = pd.read_csv("../data_clean/politicians_by_country.csv")

In [ ]:
countries_population_df = pp_df[["country", "population"]].drop_duplicates().reset_index(drop=True)
# display the 5 countries with the smallest and largest population to get an overview
countries_population_df.nlargest(5, "population").append(countries_population_df.nsmallest(5, "population"))

In [ ]:
country_population_dict = dict(zip(countries_population_df["country"], countries_population_df["population"]))

**Top 10 countries by coverage**

In [ ]:
table_1 = pp_df[["country","article_name"]].groupby(by="country", as_index=False).count()
table_1["articles/population"] = table_1.apply(lambda x: x["article_name"]/(country_population_dict[x["country"]]*1000000), axis=1)
table_1 = table_1.drop(columns=["article_name"])

table_1.nlargest(10, "articles/population").to_csv("../results/countries-coverage-top-10.csv",index=False)
table_1.nlargest(10, "articles/population").reset_index(drop=True)

**Bottom 10 countries by coverage**

In [ ]:
table_1.nsmallest(10, "articles/population").to_csv("../results/countries-coverage-bottom-10.csv",index=False)
table_1.nsmallest(10, "articles/population").reset_index(drop=True)

**Top 10 countries by relative quality**

In [ ]:
articles_per_country = dict(Counter(pp_df["country"]))
hq_articles_per_country = dict(Counter(pp_df.loc[pp_df["article_quality"].isin(["GA", "FA"])]["country"]))
countries_wo_hq_articles = pp_df.loc[~pp_df["country"].isin(hq_articles_per_country.keys())]["country"].unique()
hq_articles_per_country.update(dict(zip(countries_wo_hq_articles, np.zeros(len(countries_wo_hq_articles)))))

In [ ]:
table_2 = pd.DataFrame(pp_df["country"]).drop_duplicates()
table_2["hq_article_rate"] = table_2["country"].apply(lambda x: hq_articles_per_country[x]/articles_per_country[x])

table_2.nlargest(10, "hq_article_rate").to_csv("../results/countries-relative-quality-top-10.csv",index=False, sep=";")
table_2.nlargest(10, "hq_article_rate").reset_index(drop=True)

**Bottom 10 countries by relative quality**

In [ ]:
table_2.nsmallest(10, "hq_article_rate").to_csv("../results/countries-relative-quality-bottom-10.csv",index=False, sep=";")
table_2.nsmallest(10, "hq_article_rate").reset_index(drop=True)

**Regions by coverage**

Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population

In [ ]:
articles_per_region = dict(Counter(pp_df["region"]))
table_3 = pp_df[["population", "region"]].groupby(by="region", as_index=False).sum()
table_3["articles/population"] = table_3.apply(lambda x: articles_per_region[x["region"]]/x["population"] , axis=1)
table_3 = table_3.sort_values(by="articles/population", ascending=False)

table_3.to_csv("../results/region_coverage.csv",index=False)
table_3

**Regions by relative quality**

In [ ]:
hq_articles_per_region = dict(Counter(pp_df.loc[pp_df["article_quality"].isin(["GA", "FA"])]["region"]))

In [ ]:
table_4 = pd.DataFrame(pp_df["region"]).drop_duplicates()
table_4["hq_article_rate"] = table_4["region"].apply(lambda x: hq_articles_per_region[x]/articles_per_region[x])
table_4 = table_4.sort_values(by="hq_article_rate", ascending=False)

table_4.to_csv("../results/region_relative_quality.csv",index=False)
table_4

***

#### Credits

This exercise is slighty adapted from the course [Human Centered Data Science (Fall 2019)](https://wiki.communitydata.science/Human_Centered_Data_Science_(Fall_2019)) of [Univeristy of Washington](https://www.washington.edu/datasciencemasters/) by [Jonathan T. Morgan](https://wiki.communitydata.science/User:Jtmorgan).

Same as the original inventors, we release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).